# CLUSTER ANALYSIS

In [ ]:
# importing the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random

random.seed(123)

### K-Means

Let’s look into one of the most common clustering algorithms: K-Means in detail. Let’s see a simple example of how K-Means clustering can be used to segregate the dataset.

In this example, we'll use the `make_blobs` the command to generate isotropic gaussian blobs which can be used for clustering.
We specify the number of samples to be generated to be 100 and the number of centers to be 5.

In [ ]:
# number of clusters. make a variable as we may want to change it later
k = 5

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=100, n_features=2, centers=k, random_state=555)
X, y


In [ ]:
plt.scatter(X[:, 0], X[:, 1])

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)

As you can see from the above graph, there are 5 clusters that can be created from the dataset.

In [ ]:
y

If you look at the value of y, you can see that the points are classified based on their clusters that are predefined by using make_blobs command, we will be using this only for evaluating purpose.

For using K-Means you need to import KMeans from sklearn.cluster library.

In [ ]:
from sklearn.cluster import KMeans

For using KMeans, you need to specify a number of clusters as arguments. As we set k=5 at the beginning, we will continue with this number and see the results of step by step algorithm, and compare various methods.


In [ ]:
Cluster = KMeans(n_clusters=k, max_iter=200)
Cluster.fit(X)
y_pred = Cluster.predict(X)

After passing the arguments, we fit the model and predict the results. Now let’s visualize our predictions in a scatter plot.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_pred)

You can see that the predicted clusters are mostly the same as the clusters that you saw in the initial scatter plot. Now let’s look into how exactly the K-Means algorithm works.

## Steps Involved
There are 3 important steps in K-Means Clustering.

1. Initialize centroids – This is done by randomly choosing K no of points, the points can be present in the dataset or also random points.
2. Assign Clusters – The clusters are assigned to each point in the dataset by calculating their distance from the centroid and assigning it to the clustersroid with minimum distance.
3. Re-calculate the centroids – Updating the centroid by calculating the centroid of each cluster we have created.
Let’s look into this by an example.

Let's use the same `make_blobs` example we used at the beginning.

We will try to do the clustering without using the KMeans library.

In [ ]:
# setting the number of training examples
m=X.shape[0]
n=X.shape[1] 
n_iter=200

We will set the K value to be 5 as before and also initialize the centroids randomly using the `random.randint()` function.

In [ ]:
# creating an empty centroid array
centroids=np.array([]).reshape(n,0) 

# creating 5 random centroids
for i in range(k):
    centroids=np.c_[centroids,X[random.randint(0,m-1)]]


In [ ]:
centroids

In [ ]:
plt.scatter(X[:,0],X[:,1])
plt.scatter(centroids[0,:],centroids[1,:],s=300,c='yellow')
plt.rcParams.update({'figure.figsize':(10,7.5), 'figure.dpi':100})
plt.show()

Next, we'll find the distance between the points. Euclidean distance is most commonly used for finding the similarity.

In [ ]:
output={}

# creating an empty array
euclid=np.array([]).reshape(m,0)

# finding distance between for each centroid
for i in range(k):
       dist=np.sum((X-centroids[:,i])**2,axis=1)
       euclid=np.c_[euclid,dist]

# storing the minimum value we have computed
new_clusters=np.argmin(euclid,axis=1)+1


Each row represents a data point <br>
Each column represents the distance to a specific centroid<br>
This allows the algorithm to later find which centroid is closest to each point by taking np.argmin(euclid, axis=1).

In [ ]:
euclid

In [ ]:
new_clusters

Then we regroup the dataset based on the minimum values we got and calculate the new centroid value.

In [ ]:
# computing the mean of separated clusters
clusters={}
# creating an empty array
for i in range(k):
    clusters[i+1]=np.array([]).reshape(2,0)

# assigning clusters to the points
for i in range(m):
    clusters[new_clusters[i]]=np.c_[clusters[new_clusters[i]],X[i]]
for i in range(k):
    clusters[i+1]=clusters[i+1].T
# print(clusters)

# computing mean and updating it
for i in range(k):
     centroids[:,i]=np.mean(clusters[i+1],axis=0)
centroids

In [ ]:
clusters

In [ ]:
for i in range(k):
    plt.scatter(clusters[i+1][:,0],clusters[i+1][:,1])
plt.scatter(centroids[0,:],centroids[1,:],s=300,c='yellow')
plt.rcParams.update({'figure.figsize':(10,7.5), 'figure.dpi':100})
plt.show()

Then we need to repeat the above 2 steps over and over again until we reach the convergence.

In [ ]:
# repeating the above steps again and again
for i in range(n_iter):
      euclid=np.array([]).reshape(m,0)
      for i in range(k):
          dist=np.sum((X-centroids[:,i])**2,axis=1)
          euclid=np.c_[euclid,dist]
      C=np.argmin(euclid,axis=1)+1
      clusters={}
      for i in range(k):
           clusters[i+1]=np.array([]).reshape(2,0)
      for i in range(m):
           clusters[C[i]]=np.c_[clusters[C[i]],X[i]]
      for i in range(k):
           clusters[i+1]=clusters[i+1].T
      for i in range(k):
           centroids[:,i]=np.mean(clusters[i+1],axis=0)
      final=clusters

Let’s plot it.

In [ ]:
plt.scatter(X[:,0],X[:,1])
plt.title('Original Dataset')

This is the original dataset where it is hard to visually distinguish the clusters

In [ ]:
for i in range(k):
    plt.scatter(final[i+1][:,0],final[i+1][:,1])
plt.scatter(centroids[0,:],centroids[1,:],s=300,c='yellow')
plt.show()

In [ ]:
# the plot obtained by built-in KMeans 
plt.scatter(X[:, 0], X[:, 1], c=y_pred)

### Elbow method to find the optimal number of clusters

One of the important steps in K-Means Clustering is to determine the optimal number of clusters we need to give as an input. This can be done by iterating it through a number of n values (clusters) and then finding the optimal n value.

For finding this optimal n, the Elbow Method is used.

You have to plot the loss values vs the n value and find the point where the graph is flattening, this point is considered as the optimal n value.

Let’s look at the example we have seen at first, to see the working of the elbow method.

We will to iterate it through a series of n values ranging from 1-20 and then plot their loss values.

In [ ]:
len(X)

In [ ]:
elbow=[]
for i in range(2, 21):  # we don't need 0 and 1
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=555, max_iter=200)
    kmeans.fit(X)
    elbow.append(kmeans.inertia_)

plt.plot(range(2, 21), elbow)
plt.title('ELBOW METHOD')
plt.show()

You can see that the graph starts to flatten after reaching 5, which means that even if we increase the no of clusters after that point, there is no significant change in the loss value. So we can take the optimal value to be 5 which we also confirmed by visualizing the scatter plot.

## Average Silhouette Method to find the optimal number of clusters

The silhouette coefficient is a measure of cluster cohesion and separation. It quantifies how well a data point fits into its assigned cluster based on two factors:

* How close the data point is to other points in the cluster
* How far away the data point is from points in other clusters
Silhouette coefficient values range between -1 and 1. Larger numbers indicate that samples are closer to their clusters than they are to other clusters.

In the `scikit-learn` implementation of the silhouette coefficient, the average silhouette coefficient of all the samples is summarized into one score. The silhouette `score()` function needs a minimum of two clusters, or it will raise an exception.

Loop through values of `k` again. This time compute the silhouette coefficient:

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_coefficients = []

# Notice you start at 2 clusters for silhouette coefficient
for i in range(2, 20):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=555, max_iter=200)
    kmeans.fit(X)
    score = silhouette_score(X, kmeans.labels_)
    silhouette_coefficients.append(score)
    
# silhouette_coefficients

In [ ]:
plt.plot(range(2, 20), silhouette_coefficients)
plt.xticks(range(2, 20))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

Ultimately, your decision on the number of clusters to use should be guided by a combination of domain knowledge and clustering evaluation metrics.

# Some other examples of built in functions and clustering alghoritms. 

## Hierarchical Clustering

In [ ]:
import scipy.cluster.hierarchy as shc
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(X, method='ward'))
plt.show()

The x-axis represents our data points and y-axis - the Euclidean distance between the clusters. How to determine the optimal number of clusters from this diagram? Just visually, we can look for the largest distance (y axis) that we can vertically, without crossing any horizontal line (before splitting). The longer is the distance, the more clusters differ from each other. 
In this example, we can imagine a horizontal line above ~15. Since the distances between the corresponding clusters are rather small, we can safely draw a line at position 20 to determine the optimal number of clusters. In our case, the optimal number of clusters is between 4. This also corresponds to the elbow method, but differs from silouette coefficient method. 

In [ ]:
# Fitting hierarchical clustering to the dataset
# There are two algorithms for hierarchical clustering: Agglomerative Hierarchical Clustering and 
# Divisive Hierarchical Clustering. We choose Euclidean distance and ward method for our algorithm class

from sklearn.cluster import AgglomerativeClustering

hc = AgglomerativeClustering(n_clusters=4, metric='euclidean', linkage='ward')
y_hc = hc.fit_predict(X)

In [ ]:
y_hc

In [ ]:
for i in range(4):
    plt.scatter(X[y_hc == i,0],X[y_hc == i,1])

plt.show()

## DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

Perform DBSCAN clustering from vector array or distance matrix.

DBSCAN - Density-Based Spatial Clustering of Applications with Noise. Finds core samples of high density and expands clusters from them. Good for data which contains clusters of similar density (https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).


In [ ]:
# Let's first display again our original data
plt.scatter(X[:, 0], X[:, 1])
# setting y-axis limits for better evaluation of the distance between the points
plt.ylim((-12,12))

We will construct a DBSCAN object that requires a minimum of 4 data points in a neighborhood of radius 2.5 to be considered a core point.

In [ ]:
from sklearn.cluster import DBSCAN
# define the model
model = DBSCAN(eps=2.5, min_samples=4)
# fit model and predict clusters
yhat = model.fit_predict(X)
# retrieve unique clusters
clusters = np.unique(yhat)
clusters

In [ ]:
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(yhat == cluster)
    # create scatter of these samples
    plt.scatter(X[row_ix, 0], X[row_ix, 1])
    plt.ylim((-12,12))
    
# show the plot
plt.show()

# Principal Component Analysis (PCA)

PCA is a dimensionality reduction technique that transforms high-dimensional data into a lower-dimensional space while preserving as much variance as possible. It's useful for:
* Visualizing high-dimensional data
* Reducing computational complexity
* Removing noise and redundant features
* Data compression

PCA works by finding the principal components (directions of maximum variance) in the data.

## Example 1: PCA on the Iris Dataset

Let's start with a classic example using the Iris dataset, which has 4 features. We'll reduce it to 2 dimensions for visualization.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the Iris dataset
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print(f"Original shape: {X_iris.shape}")
print(f"Features: {iris.feature_names}")

Standardize the features before applying PCA (important when features have different scales):

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_iris_scaled = scaler.fit_transform(X_iris)

# Apply PCA to reduce to 2 components
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_iris_scaled)

print(f"Shape after PCA: {X_pca.shape}")
print(f"\nExplained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.2%}")

In [ ]:
X_pca

Visualize the data in 2D PCA space:

In [ ]:
# Create a scatter plot
plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_iris, cmap='viridis', s=50, alpha=0.7)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('PCA of Iris Dataset')
plt.colorbar(scatter, label='Species')
plt.grid(True, alpha=0.3)
plt.show()

The PCA plot of the Iris dataset shows how the 4-dimensional flower measurements are transformed into a 2-dimensional space. Here's what each element means:

Axes:

PC1 (X-axis): The first principal component - the direction in the original 4D space that captures the maximum variance in the data. The percentage shows how much of the total variance this component explains (typically ~73%).
PC2 (Y-axis): The second principal component - the direction orthogonal (perpendicular) to PC1 that captures the second-most variance (typically ~23%).
Colors: Each color represents one of the three Iris species:<br>

Setosa (one cluster)<br>
Versicolor (middle cluster)<br>
Virginica (another cluster)<br>
What it reveals:

Dimensionality Reduction: The original 4 features (sepal length, sepal width, petal length, petal width) are compressed into just 2 dimensions while retaining ~95% of the variance.

Species Separation: The plot clearly shows that:

One species (typically Setosa) is very distinct and well-separated
The other two species (Versicolor and Virginica) have some overlap but are mostly distinguishable
Clustering Structure: Even without knowing the species labels, you can visually identify 3 distinct groups, which validates that clustering algorithms would work well on this data.

## Determining the Optimal Number of Components

Use the explained variance to decide how many components to keep:

In [ ]:
# Apply PCA with all components
pca_full = PCA()
pca_full.fit(X_iris_scaled)

# Plot cumulative explained variance
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(pca_full.explained_variance_ratio_) + 1), 
         pca_full.explained_variance_ratio_, 'bo-')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(range(1, len(pca_full.explained_variance_ratio_) + 1),
         np.cumsum(pca_full.explained_variance_ratio_), 'ro-')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance')
plt.axhline(y=0.95, color='k', linestyle='--', label='95% threshold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Explained variance by each component:")
for i, var in enumerate(pca_full.explained_variance_ratio_):
    print(f"PC{i+1}: {var:.2%}")

## Visualizing Principal Components

Let's examine what the principal components actually represent:

In [ ]:
# Component loadings (how much each original feature contributes to each PC)
components_df = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=iris.feature_names
)

print("Component loadings (feature contributions to PCs):")
print(components_df)

# Visualize as heatmap
plt.figure(figsize=(8, 5))
sns.heatmap(components_df, annot=True, cmap='coolwarm', center=0, 
            cbar_kws={'label': 'Loading'}, fmt='.3f')
plt.title('PCA Component Loadings\n(How features contribute to principal components)')
plt.tight_layout()
plt.show()

### Using PCA with a Classifier to Predict Species

You can use PCA to reduce dimensionality, then train a classifier (e.g., Logistic Regression) on the PCA-transformed data. Here is an example:

In [ ]:
species_names = iris.target_names
colors = ['purple', 'green', 'yellow']
for i, name in enumerate(species_names):
    plt.scatter(X_pca[y_iris == i, 0], X_pca[y_iris == i, 1], 
                label=name, color=colors[i], s=50, alpha=0.7)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('PCA of Iris Dataset')
plt.legend(title='Species')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Use the PCA-transformed data (X_pca) and true labels (y_iris)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_iris, test_size=0.3, random_state=101)

# Train a classifier on the PCA data
clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# Use the original data (X_iris) and true labels (y_iris)
X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.3, random_state=101)

# Train a classifier on the original data
clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=iris.target_names))